# Model Testing


## Setup


In [19]:
from scripts.models.target_independent import test_independant_models
from scripts.preprocessing.data_loader import load_preprocessed_dataset, split_features_targets

from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
)
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from IPython.display import display

In [8]:
df = load_preprocessed_dataset()
features, targets = split_features_targets(df)

# Target-Independant Testing


## Refined Model Subset


### Models


In [20]:
models = [
    RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1),
    ExtraTreesRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    BaggingRegressor(n_estimators=10, max_samples=0.5, n_jobs=-1),
    XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1),
    LGBMRegressor(
        n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1, verbose=-1
    ),
]

results_thickness = test_independant_models(models, features, targets["ice_thickness"])
print("Ice Thickness Results:")
display(results_thickness)

results_velocity = test_independant_models(models, features, targets["ice_velocity"])
print("\nIce Velocity Results:")
display(results_velocity)

results_mask = test_independant_models(models, features, targets["ice_mask"])
print("\nIce Mask Results:")
display(results_mask)

Ice Thickness Results:


,MSE_mean,MAE_mean,R2_mean,MSE_std,MAE_std,R2_std
ExtraTreesRegressor,0.001591,0.019288,0.975378,0.000505,0.003714,0.007797



Ice Velocity Results:


,MSE_mean,MAE_mean,R2_mean,MSE_std,MAE_std,R2_std
ExtraTreesRegressor,1360.115072,6.43102,0.445627,1064.032219,1.53425,0.186827



Ice Mask Results:


,MSE_mean,MAE_mean,R2_mean,MSE_std,MAE_std,R2_std
ExtraTreesRegressor,0.019674,0.038448,0.978533,0.007341,0.008516,0.008015


## Further Testing


In [ ]:
MAX_DEPTH = 60
N_ESTIMATORS = 160
N_JOBS = -1

models = [
    RandomForestRegressor(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, n_jobs=N_JOBS
    ),
    GradientBoostingRegressor(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1
    ),
    ExtraTreesRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, n_jobs=N_JOBS),
    BaggingRegressor(n_estimators=N_ESTIMATORS, max_samples=0.5, n_jobs=N_JOBS),
    XGBRegressor(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1, n_jobs=N_JOBS
    ),
    LGBMRegressor(
        n_estimators=N_ESTIMATORS,
        max_depth=MAX_DEPTH,
        learning_rate=0.1,
        n_jobs=N_JOBS,
        verbose=-1,
    ),
]

print(
    "Ice Thickness\n",
    test_independant_models(
        models, features, targets["ice_thickness"], split_year=2085
    ),
)
print(
    "Ice Velocity\n",
    test_independant_models(models, features, targets["ice_velocity"], split_year=2085),
)
print(
    "Ice Mask\n",
    test_independant_models(models, features, targets["ice_mask"], split_year=2085),
)